# Use Case Generation Program


### generate usecases by searching the different data library for interested parameters and output a UseCase_set with the correct format for the automation test. 

#### Excel file: Data_Library.xlsx

#### Libraries: 
- Data with eZono machine
- Data with Toshiba machine
- Data with GE machine
- Data with SonoSite machine
- Tingting's 3D data with SonoSite machine

#### Parameters (column names):
DataFolderName	StudyName	Owner	Medium	AutomaticTankTest	ManualTankTest	US_machine	NeedleType	NeedleID	TransducerLength_mm	NeedleLength_in	PicoSeries	SamplingRate_MSps	SampleInterval_ns	SampleNum	GainSetting_dB	TestType filename	Axis0	Axis1	Axis2	Axis3	Axis4

In [360]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook

def create_usecase(data_df_list, usecase_df, lib_names, col_names, col_values): 
    """
    Goes through each library to search for the given column name and values to create a usecase information data frame.
    
    data_df_list: list of data frame objects, ea. data frame is data from 1 library
    usecase_df: data frame object of TestUseCase sheet
    lib_names: list of library names to search through. ex: ['Lib1', 'Lib2']
    col_names: list of column names to search. ex: ['Axis0', 'Axis1']
    col_values: nested list, with ea sublist contains value conditions. ex: [[60,60],[0,0]]
    return: a data frame of the usecase created
    """
    row_num = usecase_df.shape[0]+1  # next row in the TestUseCase sheet
    usecase_format=[]
    for lib in range(len(lib_names)):  # for each library
        
        # filtering normal data:
        n= [c for c in col_names if not c.startswith('Axis')]
        v= [col_values[i] for i in range(len(col_names)) if not col_names[i].startswith('Axis') ]
        filtered_lib= data_df_list[lib]
        for i,item in enumerate(n): 
            filtered_lib= filtered_lib[filtered_lib[item].isin(v[i])]
        
        # filtering different axis which can have multiple values:        
        signal_list=[]
        axiscol= [c for c in col_names if c.startswith('Axis')]
        axisval= [col_values[i] for i in range(len(col_names)) if col_names[i].startswith('Axis')]
        #filter through each set of coordinates:
        for a in range(len(axisval[0])):
            lib_df= filtered_lib
            for p in range(len(axisval)): # filter through each condition:
                condition=axisval[p][a]
                axis= axiscol[p]
                lib_df= lib_df[lib_df[axis].isin([condition])]
            signal_list= signal_list+ list(lib_df['Row'].values)    

        if signal_list==[]:continue     
        # format use case string: 
        signal_str= ",".join(str(s) for s in signal_list)  # ex: '23,45,65'
        if signal_str != '' or signal_str != ' ':  # add to list if signal is not empty
            usecase_format += ["{}: ({})".format(lib_names[lib], signal_str)]

    usecase_format= ";".join(str(s) for s in usecase_format)  # convert from list of strings to 1 string
    generated_usecase_df = pd.DataFrame([[row_num,usecase_format]],columns = ["Row","UseCase_set"])
    return generated_usecase_df


def runThis(lib_names, filename, col_names, col_values, input_row, description= 'default'):
    """
    This function will load data and conditions for the search and perform the search on the data to output an excel file of new usecases
    
    lib_names: ['Lib1', 'Lib2', 'Lib3', 'Lib4', Lib5'] #list of relevant library names to load 
    filename= 'RX_Data_Library_TH.xlsx'
    col_names= ['Axis0', 'Axis2', 'Axis3'] #list of the column names to search for 
    col_values= [[50, 50, 50],  # nested list of a set of usecases with each list corresponds to 1 column values. 
                [0, 0, 0],  # each item in a list is the condition for the corresponding column 
                [45,45, 45]]
    description: if no description str given, then the default description will includes the collumn names and its condition values
   
    """  
    # STEP 1: IMPORT DATA 
    data_df_list=[] 
    for lib in lib_names:
        data_df_list += [pd.read_excel(filename,sheet_name = lib)] # list of dataFrame objects
    usecase_df = pd.read_excel(filename,sheet_name = 'TestUseCase') 
    
    # STEp 2: ASSIGN CONDITIONS AND CREATE USECASES
    
    usecase= create_usecase(data_df_list, usecase_df, lib_names,col_names, col_values)
    if description== 'default':
        usecase['Description']= ','.join(['{}={}'.format( col_names[s], str(col_values[s]).strip('[]')) for s in range(len(col_values))])
    else:
        usecase['Description']= description
    usecase['Input_Row']= input_row    
                                          
    # STEP 3: EXPORT DATA
    
    # Method 1: save to same file: 
    book = load_workbook(filename) # open excel workbook
    sheets= book.sheetnames 
    ws= book[sheets[sheets.index('TestUseCase')]] # TestUseCase sheet
    for index, row in usecase.iterrows():    
        ws.append(row.tolist()) # add new usecase
    book.save(filename)

    print('Done! New usecase is added and saved in the same file: {}'.format(filename))
    
    # Method 2: save to a different file: 
#     current_dir= os.getcwd()
#     outfile_name= 'NewUseCaseExcel.xlsx'
#     new_usecase_df.to_excel(outfile_name, index= False)
#     print('Done! New excel file named {} has been created at {}. Should copy and paste information into the master TestUseCase to prevent overwrite of information.'.format(outfile_name, current_dir))
    
    return usecase

In [364]:
def main(): 
    """
    Collect information through user input, load the necessary information from the data excel file, and call runThis to generate the usecase.
    
    """
    # PROMPT FOR USER INPUT:

    # 1) input name of excel file
    default_file= 'RX_Data_Library_TH.xlsx'
    filename= input('Enter the data file name or press ENTER to use default file: ')
    if filename=='' or not os.path.isfile(filename): 
        print('Default data file {} will be used.\n'.format(default_file))
        filename= default_file

    # 2) input row number
    rowNum= int(input('Enter the use case input row number: '))
    input_df= pd.read_excel(filename, sheet_name= 'UseCaseGenerationInput').iloc[rowNum-1]

    # 3) input the Library 
    xls = pd.ExcelFile(filename, on_demand = True)
    sheets = xls.sheet_names
    lib_avail= [a for a in sheets if a.startswith('Lib')]
    print("\nAvailable library: {}\n".format(lib_avail))
    lib_input= input('Enter the library names or numbers, separated by comma: ')
    lib_names=[]
    for i in lib_input.split(','):
        lib_names= lib_names+ [b for b in lib_avail if i in b]

    # 4) input description
    description= input('\nEnter use case description or ENTER to use default: ')

    # EXTRACT INFORMATION FROM EXCEL FILE:
    print("\nExtracting input information...\n")
    # 1) get column names
    input_series = input_df[input_df.notnull()] #pandas series
    col_names = [a for a in input_series.index if 'Row' not in a]  # all column names with conditions

    # 2) get column values

    # Axis are allowed multiple values for usecases where needle is moving around (trajectory or in-out of plane)
    # Make sure all axis has the same number of coordinates even if they are repeating (ex: 0,0,0,0)

    axis_names = [a for a in col_names if a.startswith('Axis')]  # Axis columns

    col_values = [[input_series[col]] for col in col_names if not col.startswith('Axis')]  
    for axis in axis_names: 
        if not type(input_series[axis])== int: 
            cond_set= [int(a) for a in input_series[axis].split(',')]  # 1 set of conditions for each axis
        else: 
            cond_set= [str(input_series[axis])]
        col_values = col_values + [cond_set] # add to values list

    print("Creating use case... \n")

#     # RUN USECASE GENERATION:
#     if description=='': 
#         df= runThis(lib_names, filename, col_names, col_values, rowNum)
#     else:
#         df= runThis(lib_names, filename, col_names, col_values, rowNum, description)
#     return df 


    # STEP 1: IMPORT DATA 
    data_df_list=[] 
    for lib in lib_names:
        data_df_list += [pd.read_excel(filename,sheet_name = lib)] # list of dataFrame objects
    usecase_df = pd.read_excel(filename,sheet_name = 'TestUseCase') 
    
    # STEp 2: ASSIGN CONDITIONS AND CREATE USECASES
    
    usecase= create_usecase(data_df_list, usecase_df, lib_names,col_names, col_values)
    if description== '':
        usecase['Description']= ','.join(['{}={}'.format( col_names[s], str(col_values[s]).strip('[]')) for s in range(len(col_values))])
    else:
        usecase['Description']= description
    usecase['Input_Row']= rowNum    
                                          
    # STEP 3: EXPORT DATA
    
    # Method 1: save to same file: 
    book = load_workbook(filename) # open excel workbook
    sheets= book.sheetnames 
    ws= book[sheets[sheets.index('TestUseCase')]] # TestUseCase sheet
    for index, row in usecase.iterrows():    
        ws.append(row.tolist()) # add new usecase
    book.save(filename)

    print('Done! New usecase is added and saved in the same file: {}'.format(filename))
    return usecase

## Example 1: Creating a simple Usecase

Gives conditions for certain columns in each library and look for data that matched the conditions

    Input is Row #1 from UseCaseGnerationInput sheet. 
    Search library 1,2,3,4 for 0.6mm PZT in Tissue at 20mm away from probe and 45 deg insertion angle.

In [365]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library_TH.xlsx will be used.

Enter the use case input row number: 1

Available library: ['Lib1', 'Lib2', 'Lib3', 'Lib4', 'Lib5']

Enter the library names or numbers, separated by comma: 1,2,3,4

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library_TH.xlsx


,Row,UseCase_set,Description,Input_Row
0,1,"Lib1: (22,1015);Lib2: (67,75,1060,1068);Lib3: ...","Medium='Tissue',TransducerLength_mm=0.6,Axis0=...",1


## Example 2: Creating Trajectory

Searching the different data library for data where needle inserting (45 degrees angle insertion) starting from the upper left edge of the ultrasound machine to a lower depth toward the middle of the ultrasound machine. 

    Input is Row #2 from UseCaseGenerationInput sheet
    Search Library 1,2,3,4
    Axis 2: at 0,10,18 mm, where 0 is the middle of the US screen and 18 is the edge of the screen
    Axis 0: at 20,30,40
    Axis 3: 45 degrees insertion

In [366]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library_TH.xlsx will be used.

Enter the use case input row number: 2

Available library: ['Lib1', 'Lib2', 'Lib3', 'Lib4', 'Lib5']

Enter the library names or numbers, separated by comma: 1,2,3,4

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library_TH.xlsx


,Row,UseCase_set,Description,Input_Row
0,2,"Lib1: (219,248,303,332,4,14,24,34,43,221,250,3...","Axis0=20, 30, 40,Axis1=0, 0, 0,Axis2=18, 0, 10...",2


## Example 3: Creating In and Out of Plane Usecase
using data from Lib5: Tingting's 3D tank data with SonoSite

    Input is Row #3 from UseCaseGenerationInput sheet
    Lib5 
    Axis0: at 50mm away from probe
    Axis1: at 0,10,20mm away from center line

In [359]:
df= main()
df

Enter the data file name or press ENTER to use default file: 
Default data file RX_Data_Library_TH.xlsx will be used.

Enter the use case input row number: 3

Available library: ['Lib1', 'Lib2', 'Lib3', 'Lib4', 'Lib5']

Enter the library names or numbers, separated by comma: 5
Library to be used for usecase generation: ['Lib5']

Enter use case description or ENTER to use default: 

Extracting input information...

Creating use case... 

Done! New usecase is added and saved in the same file: RX_Data_Library_TH.xlsx


,Row,UseCase_set,Description,Input_Row
0,2,"Lib5: (3324,3334,3344)","Axis0=50, 50, 50,Axis1=0, 0, 0,Axis2=0, 10, 20",3


##  Example 4: Creating Trajectory from 3D coordinates
    
    